# Sentiment Analysis with Scikit-learn

Based on https://github.com/crawles/sentiment_analysis_twitter_model/blob/master/build-sentiment-classifier.ipynb

Using dataset from http://help.sentiment140.com/for-students/

In [ ]:
# Install bentoml library
!pip install bentoml

In [ ]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

In [ ]:
%matplotlib inline
import sys
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import bentoml

# Read Data

In [ ]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Train the model

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

In [ ]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

In [ ]:
sentiment_lr.predict([Xtest[0]])

# Export model for production

In [ ]:
%%writefile sentiment_lr_model.py
import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('sentiment_lr')])
@bentoml.env(requirements_txt='./requirements.txt')
class SentimentLRModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.sentiment_lr.predict(series)

In [ ]:
from sentiment_lr_model import SentimentLRModel

# Initialize bentoML model with artifacts

bento_model = SentimentLRModel.pack(
    sentiment_lr=sentiment_lr
)

# Save bentoML model to directory
saved_path = bento_model.save("/tmp/bento")

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)

# Load model from archive

In [ ]:
import bentoml

# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)

# Call predict on the restored sklearn model
bento_model.predict(pd.Series(["hello", "hi"]))

##### * For demo purpurse, copy generated model to ./model folder

In [ ]:
import os
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

# Deploy to AWS lambda

```bash
bentoml deploy ARCHIVE_PATH --platform=PLATFORM --region=REGION --stage=STAGE
```
### Arguments:
* archive_path: The file path or s3 that contains BentoML bundles.

### Options:
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.
* stage: OPTIONAL Stage is a helper value that identify the position in the workflow process. DEFAULT value is dev


In [ ]:
!bentoml deploy ./model --platform aws-lambda --region us-west-2

### Make curl request to the Lambda endpoint

Update the URL from previous command's return result and run following command in the terminal
```bash
curl -i \
--header "Content-Type: application/json" \
--data '["good movie", "bad food"]' \
--request POST \
https://URL
```

# Check deployment status

```
bentoml checkt-deployment-status ARCHIVE_PATH --platform=PLATFORM --region=REGION --stage=STAGE
```

### Arguments:
* archive_path: The file path or s3 that contains BentoML bundles.

### Options:
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.
* stage: OPTIONAL Stage is a helper value that identify the position in the workflow process. DEFAULT value is dev

In [ ]:
!bentoml check-deployment-status ./model --platform=aws-lambda --region=us-west-2

# Delete serverless Deployment


```bash
bentoml delete-deployment ARCHIVE_PATH --platform=PLATFORM
```
### Arguments:
* archive_path: The file path or s3 that contains BentoML bundles.

### Options:
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.
* stage: OPTIONAL Stage is a helper value that identify the position in the workflow process. DEFAULT value is dev



In [ ]:
!bentoml delete-deployment ./model --platform aws-lambda --region us-west-2